In [ ]:
# %%
import google.generativeai as genai
import time
import threading
import json
import os
from pathlib import Path
from tqdm import tqdm
import tempfile
import chardet
import zipfile
import shutil
import concurrent.futures
import re
import openai
import requests

# %%
# Initialize the thread lock
lock = threading.Lock()

# %%
# Configuration for Gemini
gemini_model = None
def run_gemini_query(prompt, history, completion_tokens, temp, model):
    global gemini_model

    tries = 0
    while tries < 30:
        tries += 1
        try:
            if not gemini_model:
                genai.configure(api_key="")            
                gemini_model = genai.GenerativeModel(model)

            safety_settings = [
                {"category": "HARM_CATEGORY_HARASSMENT", "threshold": "BLOCK_NONE"},
                {"category": "HARM_CATEGORY_HATE_SPEECH", "threshold": "BLOCK_NONE"},
                {"category": "HARM_CATEGORY_SEXUALLY_EXPLICIT", "threshold": "BLOCK_NONE"},
                {"category": "HARM_CATEGORY_DANGEROUS_CONTENT", "threshold": "BLOCK_NONE"},
            ]

            response = gemini_model.generate_content(
                prompt,
                generation_config=genai.types.GenerationConfig(
                    candidate_count=1,
                    max_output_tokens=completion_tokens,
                    temperature=temp
                ),
                safety_settings=safety_settings
            )

            input_toks = response.usage_metadata.prompt_token_count
            output_toks = response.usage_metadata.candidates_token_count
            cost_usd = input_toks / 1_000_000 * 1.25 + output_toks / 1_000_000 * 5

            # Update costs.json with locking
            with lock:
                if os.path.exists('gemini_costs.json'):
                    with open('gemini_costs.json', 'r') as f:
                        costs = json.load(f)
                else:
                    costs = {}
                costs[model] = costs.get(model, 0) + cost_usd
                with open('gemini_costs.json', 'w') as f:
                    json.dump(costs, f)

            inference = response.text.strip() if response.text else None

            if inference:
                return inference
            else:
                print('Error: message is empty')
                time.sleep(5)

        except Exception as e:
            print("Request failed.")
            print(e)
            if str(e) != '429 Resource has been exhausted (e.g. check quota).':
                return None
            print('retrying', tries)
            time.sleep(20)

    return None


# %%
# Function to parse scenes from text
def parse_scenes_from_text(text: str):
    try:
        scene_pattern = re.compile(r'## SCENE \d+:\n(.+?)(?=(?:## SCENE \d+:|$))', re.DOTALL)
        scenes = scene_pattern.findall(text)
        return [scene.strip() for scene in scenes] if scenes else []
    except Exception as e:
        return []

# %%
# Selection prompt template
selection_prompt = """
[TEXT START]
<TEXT>
[TEXT END]

Your task is to examine the above **public domain** work for evocative, interesting, well-written scenes.

Instructions:

- Choose 3 long scenes of approx 1000-1500 words, and output them in their entirety.
- You must also repair paragraphs by restoring sentences within a paragraph that are broken by a newline.
- Double newline between paragraphs.
- Do not include chapter names.

Output in this format:

## THOUGHT PROCESS:

<thought process for the scene selection>

## SCENE 1:

<the text of approx 1000-1500 words>

## SCENE 2:

<the text of approx 1000-1500 words>

## SCENE 3:

<the text of approx 1000-1500 words>


--

Output precisely in this format. Do not add any additional commentary or explanations.
"""

# %%
# Function to detect file encoding
def detect_encoding(file_path: Path):
    try:
        with open(file_path, 'rb') as f:
            raw_data = f.read(100000)  # Read first 100KB for detection
        result = chardet.detect(raw_data)
        encoding = result['encoding']
        confidence = result['confidence']
        if encoding:
            encoding_lower = encoding.lower()
            if encoding_lower in ['ascii', 'charmap']:
                return 'latin-1'
            if confidence >= 0.5:
                return encoding
        return 'utf-8'  # Fallback encoding
    except Exception as e:
        print(f"Error detecting encoding for '{file_path}': {e}. Falling back to 'utf-8'.")
        return 'utf-8'  # Fallback encoding

# %%
# Worker function to process a single file
def process_file(key, selection, results_path, selection_prompt):
    # Check if file already processed
    with lock:
        if os.path.exists(results_path):
            with open(results_path, 'r') as f:
                scenes = json.load(f)
        else:
            scenes = {}
        if key in scenes:
            print(f"Skipping '{key}': already processed.")
            return

    response = requests.get(selection["text_url"])

    content = response.text

    if len(content) < 10000:
        print(f"Skipping '{key}': content too small!")
        return
    
    if content.find('End of the Project Gutenberg'):
        content = content[:content.find('End of the Project Gutenberg')].strip()
    else:
        print('!! not found')

    max_seg_length = 400000
    # Split into 4 even segments, ensuring max length
    #segment_length = max_seg_length
    #segments = [content[i:i+segment_length] for i in range(0, len(content), segment_length)]
    #segments = segments[:4]  # Limit to first 4 segments
    segments = [content[i:i+len(content)//4][:max_seg_length] for i in range(0, len(content), len(content)//4)]
    segments = segments[:4]

    # Collect scenes from all segments
    all_scenes = []
    for i, segment in enumerate(segments):
        prompt = selection_prompt
        if i == 0:
            prompt += "\nThe first scene you select should be the very start of the book. The other two can be selected from anywhere."

        prompt = prompt.replace('<TEXT>', segment)

        result = run_gemini_query(prompt, [], 16000, 0, "gemini-1.5-pro-002")

        #print('-'*10)
        #print(result)
        #print('-'*10)

        if result:
            this_scenes = parse_scenes_from_text(result)
            print(len(this_scenes))
            #if not scenes:
            #    print('!!')

            all_scenes.extend(this_scenes)
        else:
            print(f"Failed to get scenes for '{key}' segment {i+1}.")

        

    # Update scenes.json with locking
    with lock:
        if os.path.exists(results_path):
            with open(results_path, 'r') as f:
                scenes = json.load(f)
        else:
            scenes = {}
        if key not in scenes:
            scenes[key] = []
        scenes[key].extend(all_scenes)
        with open(results_path, 'w') as f:
            json.dump(scenes, f, indent=2)


In [8]:
import json

# %%
# Main function to orchestrate multithreading
def main():
    #filenames_path = 'data/fiction_book_selections.json'
    #filenames_path = 'data/fantasy_book_selections.json'
    #filenames_path = 'data/scifi_book_selections.json'
    selections_path = 'data/site/site_fiction_final_selections.json'

    #results_path = "scenes.json"
    #results_path = "scenes_fantasy.json"
    #results_path = "scenes_scifi.json"
    results_path = "data/site/processing/site_scenes_fiction.json"
    num_threads = 32

    # Load filenames
    with open(selections_path, 'r') as f:
        selections = json.load(f)

    #print(len(filenames))
    
    if True:
        with open('data/site/site_fantasy_final_selections.json', 'r') as f:
            already_selected = json.load(f)

        for k in already_selected:
            try:
                del selections[k]
            except:
                pass


    if True:
        with open('data/site/site_scifi_final_selections.json', 'r') as f:
            already_selected = json.load(f)

        for k in already_selected:
            try:
                del selections[k]
            except:
                pass

    if True:
        with open('data/site/site_adventure_final_selections.json', 'r') as f:
            already_selected = json.load(f)

        for k in already_selected:
            try:
                del selections[k]
            except:
                pass

    

    # Initialize tqdm
    progress = tqdm(total=len(selections), desc="Processing files")

    # Define a helper to update tqdm
    def update_progress(future):
        progress.update(1)
        exception = future.exception()
        if exception:
            print(f"Error occurred: {exception}")

    # Use ThreadPoolExecutor for multithreading
    with concurrent.futures.ThreadPoolExecutor(max_workers=num_threads) as executor:
        futures = []
        for k,v in selections.items():
            future = executor.submit(process_file, k, v, results_path, selection_prompt)
            future.add_done_callback(update_progress)
            futures.append(future)
            time.sleep(5)

        # Wait for all futures to complete
        concurrent.futures.wait(futures)

    progress.close()
    print("Processing completed.")


In [9]:
main()

Processing files:   0%|          | 0/64 [00:00<?, ?it/s]

Request failed.
Invalid operation: The `response.text` quick accessor requires the response to contain a valid `Part`, but none were returned. The candidate's [finish_reason](https://ai.google.dev/api/generate-content#finishreason) is 4. Meaning that the model was reciting from copyrighted material.
Failed to get scenes for 'Mary Wollstonecraft Shelley -- Frankenstein; Or, The Modern Prometheus' segment 1.
3
Request failed.
Invalid operation: The `response.text` quick accessor requires the response to contain a valid `Part`, but none were returned. The candidate's [finish_reason](https://ai.google.dev/api/generate-content#finishreason) is 4. Meaning that the model was reciting from copyrighted material.
Failed to get scenes for 'Charlotte Perkins Gilman -- The Yellow Wallpaper' segment 1.
3
3
3
3
Request failed.
Invalid operation: The `response.text` quick accessor requires the response to contain a valid `Part`, but none were returned. The candidate's [finish_reason](https://ai.google

Processing files:   2%|▏         | 1/64 [07:56<8:20:43, 476.89s/it]

3
3


Processing files:   3%|▎         | 2/64 [08:10<3:31:11, 204.38s/it]

...

3


Processing files:  98%|█████████▊| 63/64 [32:57<00:20, 20.02s/it]

3


Processing files: 100%|██████████| 64/64 [33:10<00:00, 31.10s/it]

3
Processing completed.
